# Experiment Notebook: Network Issuance and Inflation Rate

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis: Inflation Rate and ETH Supply Over Time](#Analysis:-Inflation-Rate-and-ETH-Supply-Over-Time)

# Experiment Summary 

The purpose of this notebook is to explore the ETH issuance and resulting annualized inflation rate across different time horizons, adoption scenarios, and network-upgrade stages for both historical data and simulated states.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
import setup

import copy
import logging
import IPython
import numpy as np
import pandas as pd
from datetime import datetime

from experiments.notebooks import visualizations
from experiments.notebooks.visualizations.eth_supply_simulator import run_eth_supply_simulator
from experiments.run import run
from model.types import Stage
from data.historical_values import df_ether_supply

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False
logging.getLogger('werkzeug').setLevel(logging.ERROR)
logging.getLogger('urllib3').setLevel(logging.ERROR)

In [ ]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

In [ ]:
# Fetch the time-domain analysis experiment
experiment = time_domain_analysis.experiment
# Create a copy of the experiment simulation
simulation = copy.deepcopy(experiment.simulations[0])

In [ ]:
# Experiment configuration

simulation_names = {
    'Validator Adoption Scenarios': [
        'Normal Adoption',
        'Low Adoption',
        'High Adoption'
    ],
    'PoS Activation Date Scenarios': [
        "2021/12/1",
        "2022/03/1",
        "2022/06/1",
    ],
    'EIP-1559 Scenarios': [
        'Disabled (Base Fee = 0)',
        'Enabled (Base Fee = 30)',
    ]
}

simulation_1 = copy.deepcopy(simulation)
normal_adoption = simulation_1.model.params['validator_process'][0](_run=None, _timestep=None)
simulation_1.model.params.update({
    'validator_process': [
        lambda _run, _timestep: normal_adoption,  # Normal adoption: current 6-month average active validators per epoch from The Graph Subgraph
        lambda _run, _timestep: normal_adoption * 0.5,  # Low adoption: 50%-lower scenario
        lambda _run, _timestep: normal_adoption * 1.5,  # High adoption: 50%-higher scenario
    ],
})

simulation_2 = copy.deepcopy(simulation)
simulation_2.model.params.update({
    'date_pos': [
        datetime.strptime("2021/12/1", "%Y/%m/%d"),
        datetime.strptime("2022/03/1", "%Y/%m/%d"),
        datetime.strptime("2022/06/1", "%Y/%m/%d"),
    ],
})

simulation_3 = copy.deepcopy(simulation)
simulation_3.model.params.update({
    'base_fee_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 30, # Enabled: see ASSUMPTIONS.md for default value assumption
    ],  # Gwei per gas
})

experiment.simulations = [
    simulation_1,
    simulation_2,
    simulation_3
]

In [ ]:
df, _exceptions = run(experiment)

# Analysis: Inflation Rate and ETH Supply Over Time

This analysis enables the exploration of the inflation rate and ETH supply over time, and supports the three adoption scenarios introduced in the second experiment notebook, as well as custom parameter choices for the Proof-of-Stake (PoS) Activation Date ("The Merge") and EIP-1559 Base Fee.

Default scenarios were selected for each of the Validator Adoption, Proof-of-Stake Activation Date, and EIP-1559 categories:
* Validator Adoption (as seen in Notebook "Validator Revenue and Profit Yields")
    * Normal adoption: assumes an average of 3 new validators per epoch. These rates correspond to the historical average newly **activated** validators per epoch between 15 January 2021 and 15 July 2021 as per [Beaconscan](https://beaconscan.com/stat/validator).
    * Low adoption: assumes an average of 1.5 new validators per epoch, i.e. a 50% lower rate compared to the base scenario
    * High adoption: assumes an average of 4.5 new validators per epoch, i.e. a 50% higher rate compared to the base scenario
* Proof-of-Stake Activation Date
    * Scenarios starting from 1 Decemeber 2021 in quarterly increments 
* EIP-1559 Base Fee (Gwei per gas)
    * EIP-1559 Disabled: Base Fee 0
    * EIP-1559 Enabled: Base Fee 30 (see [assumptions doc](../../ASSUMPTIONS.md) for default value assumption)

The first chart ("Inflation Rate and ETH Supply Analysis Scenarios") depicts the ETH supply for default scenarios of the Validator Adoption, Proof-of-Stake Activation Date, and EIP-1559 scenario categories, side by side (choose via button selector). This allows comparative analysis for each category.

We can interpret that:
* Increased Validator Adoption (i.e. implied ETH staked over time) results in higher inflation due to increased issuance for validator rewards
* A delay in the Proof-of-Stake Activation Date results in a higher peak ETH supply due to Proof-of-Work issuance being significantly higher than that of Proof-of-Stake
* EIP-1559 burns the base fee, which in certain conditions results in a deflationary ETH supply

The second chart ("Inflation Rate and ETH Supply Over Time") depicts the historical inflation rate and ETH supply over time alongside the simulated projections of the inflation rate and ETH supply. Various historical and simulated milestones are included for context, such as the "Homestead" hard-fork causing a temporary increase in the inflation rate, or the simulated effect of EIP-1559 being enabled and Proof-of-Stake being activated. The interface allows us to both select the default scenarios defined above, or customize each parameter for unique analyses.

In [ ]:
visualizations.plot_network_issuance_scenarios(df, simulation_names)

In [ ]:
logger.disabled = True
run_eth_supply_simulator(execution_mode='inline')

# To display in new browser tab at http://127.0.0.1:8050/:
# run_eth_supply_simulator(execution_mode='external')

# To display either in "inline" mode when using Jupyter Notebook,
# or "jupyterlab" mode when using Jupyter Lab:
# run_eth_supply_simulator()